In [76]:
from typing import Callable,Dict, Literal, Optional
from langchain_ollama import ChatOllama
import re 
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import uuid

In [77]:
def booking_handler(request: str) -> str:
    """
    Handles booking requests for flights and hotels.
    """
    print("-------------------------- Booking Handler Called ----------------------------")
    return f"Booking action for '{request}' has been simulated."

def info_handler(request: str) -> str:
    """
    Handles general information requests.
    """
    print("-------------------------- Info Handler Called ----------------------------")
    return f"Information request for '{request}'. Result: Simulated information retrieval."

def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    return f"Coordinator could not delegate request: '{request}'. Please clarify."

In [78]:
booking_handler("some request")

-------------------------- Booking Handler Called ----------------------------


"Booking action for 'some request' has been simulated."

In [79]:
# -- Minimal Agent Abstraction -------------
class Agent:
    def __init__(self, name:str, description:str,tool: Callable[[str],str]):
        self.name = name
        self.description = description
        self.tool = tool
    
    def run(self,request:str) -> str:
        return self.tool(request)
    
booking_agent = Agent(
    name = "Booker",
    description= "Handles all the flight and hotel booking request",
    tool = booking_handler,
)

info_agent = Agent(
    name= "Info",
    description= "Provdies general information and answers user questions",
    tool= info_handler,
)

In [80]:
booking_agent.run("my requst")

-------------------------- Booking Handler Called ----------------------------


"Booking action for 'my requst' has been simulated."

In [81]:
booking_agent.name

'Booker'

In [82]:
AGENT_REGISTRY: Dict[str, Agent] = {
    "Booker": booking_agent,
    "Info": info_agent,
}

In [83]:
# LLM Setup

llm_text = ChatOllama(
    model= "deepseek-r1:8b",
    temperature=0,
)

llm_json = ChatOllama(
    model="deepseek-r1:8b",
    temperature=0,
    format = "json",
)

def strip_think(text:str) -> str:
    """
    Remove any < think> blocks
    """
    
    return re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL | re.IGNORECASE)
    

In [84]:
# to invoke chat using llm
messages = [
    ("human", "Tell me about yourself")
]

In [85]:
chat_text = llm_text.invoke(messages)

In [86]:
chat_text.content

"<think>\nOkay, the user asked “Tell me about yourself” – pretty common opening question! They're probably just starting to get familiar with me and want a general overview. \n\nHmm, this seems like a broad request but also an opportunity to set a friendly tone right from the start. The user might be curious about my capabilities or just testing how I respond. Since they didn't specify any particular topic, I should keep it balanced – cover what I am (AI assistant), what I do (helpful tasks), and who I work for (DeepSeek) without going too technical.\n\nI notice they used simple English with no special formatting requests, so I'll match that casual but clear style. The exclamation points in my response feel appropriate to show enthusiasm – this isn't a cold corporate introduction after all! \n\nThe “just your friendly AI assistant” line feels right because it humanizes me without being misleading. Mentioning the model name (DeepSeek R1) adds credibility subtly, while keeping it simple 

In [87]:
strip_think(chat_text.content)

"Sure! 😊\n\nI'm your friendly AI assistant, powered by **DeepSeek R1**, and I’m here to help you with anything from answering questions and solving problems to brainstorming ideas, writing content, learning new things, or just having a fun conversation. My goal is to be helpful, accurate, and easy to understand.\n\nI was created by **DeepSeek**, an AI company focused on building advanced language models that can assist people in creative, educational, and practical ways — like you're talking to another smart friend who knows how to help!\n\nWant to know more about me? Or maybe we should jump right into something you need help with? 😊"

In [88]:
chat_json = llm_json.invoke(messages)
chat_json

AIMessage(content='{\n\n\n  "role": "assistant",\n  "model": "gpt-3.5-turbo-0125",\n  "objectives": [\n    {\n      "difficulty": "easy",\n      "description": "Give a brief introduction of the assistant.",\n      "isCompleted": true\n    }\n  ],\n  "contextLength": 1000,\n  "totalTokenCount": 369,\n  "completionTokenCount": 285,\n  "promptTokenCount": 84,\n  "detectedContentTopics": [\n    "Technology"\n  ]\n}', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-10-09T14:03:09.964270995Z', 'done': True, 'done_reason': 'stop', 'total_duration': 33091585292, 'load_duration': 343127297, 'prompt_eval_count': 6, 'prompt_eval_duration': 345426456, 'eval_count': 126, 'eval_duration': 32393173097, 'model_name': 'deepseek-r1:8b'}, id='run--acf428a8-374b-4055-b5b0-c82c574d22b4-0', usage_metadata={'input_tokens': 6, 'output_tokens': 126, 'total_tokens': 132})

In [89]:
print(chat_json.content)

{


  "role": "assistant",
  "model": "gpt-3.5-turbo-0125",
  "objectives": [
    {
      "difficulty": "easy",
      "description": "Give a brief introduction of the assistant.",
      "isCompleted": true
    }
  ],
  "contextLength": 1000,
  "totalTokenCount": 369,
  "completionTokenCount": 285,
  "promptTokenCount": 84,
  "detectedContentTopics": [
    "Technology"
  ]
}


In [90]:
# Implementing Routing

class RouteDecision(BaseModel):
    route: Literal["Booker", "Info", "Unclear"] = Field(
        description="Which sub-agent should handle this request: 'Booker' for booking flights/hotels; 'Info' for general questions; 'Unclear' if ambiguous."
    )
    reason: str = Field(description="Brief reason for the decision.")

In [91]:
test_route_decsion = RouteDecision(route= "Booker",reason="not")

In [92]:
test_route_decsion.route

'Booker'

In [93]:
ROUTER_SYSTEM = """You are the Coordinator. Your ONLY task:
- Analyze the user's request and choose the proper sub-agent.
- Never answer the user directly.
- Return STRICT JSON with keys: route, reason.

Routing rules:
- If the request is about booking flights or hotels -> route = "Booker".
- Otherwise, if it's a general information question -> route = "Info".
- If it's ambiguous or cannot be routed -> route = "Unclear".
"""

ROUTER_HUMAN = """User request:
{request}

Return JSON only, like:
{{"route": "Booker", "reason": "It's about hotel booking"}}
"""

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ROUTER_SYSTEM),
        ("human", ROUTER_HUMAN),
    ]
)


In [94]:
router_chain = router_prompt | llm_json|JsonOutputParser(pydantic_object=RouteDecision)


In [95]:
class Coordinator:
    def __init__(self):
        self.session_id = str(uuid.uuid4())

    def route(self, request: str) -> RouteDecision:
        decision: RouteDecision = router_chain.invoke({"request": request})
        return decision

    def handle(self, request: str) -> str:
        print(f"\n--- Coordinator handling request: '{request}' ---")
        try:
            decision = self.route(request)
            print(f"Routing -> {decision["route"]} | Reason: {decision["reason"]}")

            if decision["route"] in AGENT_REGISTRY:
                agent = AGENT_REGISTRY[decision["route"]]
                return agent.run(request)
            else:
                return unclear_handler(request)
        except Exception as e:
            return f"An error occurred while processing your request: {e}"
    
    

In [ ]:
coordinator = Coordinator()
c = coordinator.handle("Book me a hotel in Paris.")


--- Coordinator handling request: 'Book me a hotel in Paris.' ---
Routing -> Booker | Reason: The user is explicitly asking to book a hotel in Paris.
-------------------------- Booking Handler Called ----------------------------


In [97]:
def main():
    print("--- Ollama deepseek-r1 Routing Example (Auto-Flow Style) ---")
    print("Note: Requires Ollama running locally with `deepseek-r1:7b` pulled.\n")

    coordinator = Coordinator()

    # Example Usage
    result_a = coordinator.handle("Book me a hotel in Paris.")
    print(f"Final Output A: {result_a}")

    result_b = coordinator.handle("What is the highest mountain in the world?")
    print(f"Final Output B: {result_b}")

    result_c = coordinator.handle("Tell me a random fact.")  # Should go to Info
    print(f"Final Output C: {result_c}")

    result_d = coordinator.handle("Find flights to Tokyo next month.")  # Should go to Booker
    print(f"Final Output D: {result_d}")


if __name__ == "__main__":
    main()

--- Ollama deepseek-r1 Routing Example (Auto-Flow Style) ---
Note: Requires Ollama running locally with `deepseek-r1:7b` pulled.


--- Coordinator handling request: 'Book me a hotel in Paris.' ---
Routing -> Booker | Reason: The user is explicitly asking to book a hotel in Paris, which falls under the category of booking requests.
-------------------------- Booking Handler Called ----------------------------
Final Output A: Booking action for 'Book me a hotel in Paris.' has been simulated.

--- Coordinator handling request: 'What is the highest mountain in the world?' ---
Routing -> Unclear | Reason: The user asked for information about mountains but did not specify if they want to book a flight or hotel. The request does not clearly match any of the predefined routes.
Final Output B: Coordinator could not delegate request: 'What is the highest mountain in the world?'. Please clarify.

--- Coordinator handling request: 'Tell me a random fact.' ---
Routing -> Unclear | Reason: The user 